In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()
data = spark.read.csv('/FileStore/tables/Bike_Share.csv',inferSchema=True,header=True)
data.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: integer (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

In [2]:
data.describe('cnt').show()

+-------+------------------+
summary| cnt|
+-------+------------------+
 count| 17379|
 mean|189.46308763450142|
 stddev| 181.3875990918646|
 min| 1|
 max| 977|
+-------+------------------+

In [3]:
#cnt = casual + registered
#season and dteday are coverd by yr, mnth, and weekday
#holiday and weekday are highly correlated with workingday
#instant is row index column
#atemp is highly correlated with temp
data = data.drop("casual", "registered", "dteday", "season", "holiday", "weekday", "instant", "atemp")
data.printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)

In [4]:
from pyspark.mllib.stat import Statistics
import pandas as pd
col_names = data.columns
features = data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names
print(corr_df.to_string())

yr mnth hr workingday weathersit temp hum windspeed cnt
yr 1.000000 -0.010473 -0.003867 -0.002196 -0.019157 0.040913 -0.083546 -0.008740 0.250495
mnth -0.010473 1.000000 -0.005772 -0.003477 0.005400 0.201691 0.164411 -0.135386 0.120638
hr -0.003867 -0.005772 1.000000 0.002285 -0.020203 0.137603 -0.276498 0.137252 0.394071
workingday -0.002196 -0.003477 0.002285 1.000000 0.044672 0.055390 0.015688 -0.011830 0.030284
weathersit -0.019157 0.005400 -0.020203 0.044672 1.000000 -0.102640 0.418130 0.026226 -0.142426
temp 0.040913 0.201691 0.137603 0.055390 -0.102640 1.000000 -0.069881 -0.023125 0.404772
hum -0.083546 0.164411 -0.276498 0.015688 0.418130 -0.069881 1.000000 -0.290105 -0.322911
windspeed -0.008740 -0.135386 0.137252 -0.011830 0.026226 -0.023125 -0.290105 1.000000 0.093234
cnt 0.250495 0.120638 0.394071 0.030284 -0.142426 0.404772 -0.322911 0.093234 1.000000

In [5]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['yr','mnth','hr','workingday','weathersit','temp','hum','windspeed'],outputCol='features')
output = assembler.transform(data)
final_data = output.select('features','cnt')

In [6]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.0, labelCol='cnt')
model = glr.fit(final_data)
summary = model.summary
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("AIC: " + str(summary.aic))

Coefficients: [80.6070235728,5.08988834706,7.65591400666,6.5609103192,-4.20164789921,301.015773794,-194.191083747,24.0796769961]
Intercept: -3.591256782869093
Coefficient Standard Errors: [2.175166632995379, 0.3282379138190242, 0.16557195957694057, 2.3308634070201713, 1.9114877733884503, 5.844330571793608, 6.901165784363383, 9.425643371396651, 6.5196975954424286]
T Values: [37.057861384062, 15.506704535863488, 46.23919428276566, 2.814798284376211, -2.198103465637391, 51.50560360947058, -28.13888114200394, 2.5546985014452677, -0.5508318032081041]
P Values: [0.0, 0.0, 0.0, 0.004886277319453569, 0.027954878766552227, 0.0, 0.0, 0.010636431434967397, 0.5817560647532338]
AIC: 221724.7008802438

In [7]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.0, labelCol='cnt', fitIntercept=False)
model = glr.fit(final_data)
summary = model.summary
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("AIC: " + str(summary.aic))

Coefficients: [80.3412224847,5.0587763955,7.62392125749,6.28986934261,-4.28002531174,299.803176474,-196.518508615,21.8344372674]
Intercept: 0.0
Coefficient Standard Errors: [2.1209243009248273, 0.323335346012136, 0.15504751201328415, 2.27828781860648, 1.906146265912169, 5.4137688346198125, 5.456354832851995, 8.498549933935461]
T Values: [37.88028759424551, 15.64560280184728, 49.17151625647387, 2.7607878562307886, -2.2453813688257, 55.377905047704665, -36.016445893866255, 2.5691956200909556]
P Values: [0.0, 0.0, 0.0, 0.005772261265023726, 0.0247561843021602, 0.0, 0.0, 0.010201758509832404]
AIC: 221723.004450478

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
my_eval = RegressionEvaluator().setLabelCol("cnt")
results = model.transform(final_data)
results.describe('prediction').show()

+-------+-------------------+
summary| prediction|
+-------+-------------------+
 count| 17379|
 mean| 189.56190471767115|
 stddev| 112.01063820342674|
 min|-120.76132152786755|
 max| 499.50524649746524|
+-------+-------------------+

In [9]:
RMSE = my_eval.evaluate(results)
RMSE

Out[ 9 ]: 142.53579425441478